# 0 Introduction

Create a better classification model to classify images

# Content Page

1. Import
2. Preprocessing
3. Define Model
4. Train Model
5. Output

# 1 Import

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
import pickle
import tensorflow as tf


from keras.layers import Input
from keras.models import Model
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

Using TensorFlow backend.


# 2 Preprocessing

Get processed images to be used as dataset to train model

In [2]:
IMG_SIZE = 50

pickle_in = open("X_improved.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y_improved.pickle","rb")
y = pickle.load(pickle_in)

# 3 Define Model

Create a model using a pre-trained model (VGG16) with 3 added layers at the end - Flatten, Dense64 and Dense5

In [3]:
image_input = Input(shape=(IMG_SIZE,IMG_SIZE,3))

model = VGG16(input_tensor=image_input, include_top=False, weights='imagenet')

In [4]:
#model.summary()
#model.add(Flatten())
#model.add(Dense(64))
#model.add(Activation("relu"))

#third_last_layer = model.get_layer('block5_pool').output
#out = Flatten(name='flatten')(third_last_layer)
#custom_vgg_model = Model(image_input, out)

#second_last_layer = custom_vgg_model.get_layer('flatten').output
#out = Dense(64,activation='relu', name='dense64')(second_last_layer)
#custom_vgg_model = Model(image_input, out)

#last_layer = custom_vgg_model.get_layer('dense64').output
#out = Dense(5,activation='softmax', name='output')(last_layer)
#custom_vgg_model = Model(image_input, out)

last_layer = model.get_layer('block5_pool').output
tmp = Flatten(name='flatten')(last_layer)
tmp = Dense(64,activation='relu',name='dense64')(tmp)
out = Dense(5,activation='softmax', name='output')(tmp)
custom_vgg_model = Model(image_input, out)

In [5]:
custom_vgg_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50, 50, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 50, 50, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 50, 50, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 25, 25, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 25, 25, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 25, 25, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0   

In [6]:
custom_vgg_model.compile(loss='sparse_categorical_crossentropy',
                         optimizer='adadelta',
                         metrics=['accuracy'])

# 4 Train Model

Train model with all layers freezed except for last three added layers. Providing a better classification with an accuracy between 65-75%

In [7]:
X = X/255.0

for layer in custom_vgg_model.layers[:-3]:
    layer.trainable = False
    
custom_vgg_model.compile(loss='sparse_categorical_crossentropy',
                         optimizer='adadelta',
                         metrics=['accuracy'])    
    
history = custom_vgg_model.fit(X,y,epochs=12, validation_split=0.1) 

Train on 3890 samples, validate on 433 samples
Epoch 1/12
3890/3890 [==============================] - 73s 19ms/step - loss: 1.1089 - accuracy: 0.5728 - val_loss: 1.0042 - val_accuracy: 0.6005
Epoch 2/12
3890/3890 [==============================] - 66s 17ms/step - loss: 0.8416 - accuracy: 0.6730 - val_loss: 0.8589 - val_accuracy: 0.6420
Epoch 3/12
3890/3890 [==============================] - 64s 16ms/step - loss: 0.7829 - accuracy: 0.6997 - val_loss: 0.8367 - val_accuracy: 0.6651
Epoch 4/12
3890/3890 [==============================] - 67s 17ms/step - loss: 0.7354 - accuracy: 0.7224 - val_loss: 0.8093 - val_accuracy: 0.6813
Epoch 5/12
3890/3890 [==============================] - 71s 18ms/step - loss: 0.7040 - accuracy: 0.7339 - val_loss: 0.7999 - val_accuracy: 0.6859
Epoch 6/12
3890/3890 [==============================] - 64s 16ms/step - loss: 0.6772 - accuracy: 0.7445 - val_loss: 0.8544 - val_accuracy: 0.6559
Epoch 7/12
3890/3890 [==============================] - 64s 17ms/step - loss:

# 5 Output

Export Model to be used for another file or at another time

In [8]:
custom_vgg_model.save("improved_classification.model")